In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import glob
from pathlib import Path

import ta

import yfinance as yf
import nasdaqdatalink
import pandas_datareader.famafrench as ff
import pandas_datareader.data as web

from keys import nasdaq_api_key
nasdaqdatalink.ApiConfig.api_key = nasdaq_api_key

https://www.nasdaq.com/market-activity/stocks/screener

In [ ]:
nasdaq = pd.read_csv("data/symbols.csv")
symbols = nasdaq["Symbol"].to_list()

start_date = "1980-01-02"
end_date = "2023-04-01"


raw = yf.download(
    tickers=symbols,
    start=start_date,
    end=end_date,
    group_by="ticker",
    auto_adjust=True,
    threads=True,
)

# failed_symbols = []
# symbols_healthy = [item for item in symbols if item not in failed]

for symb in symbols:
    single_df = raw.xs(symb, level=0, axis=1)
    single_df.to_csv("data/stockdata_bigcap/" + symb + ".csv")


In [ ]:
nasdaq = pd.read_csv("data/symbolssmall.csv")
symbols = nasdaq["Symbol"].to_list()

start_date = "1980-01-02"
end_date = "2023-04-01"


raw = yf.download(
    tickers=symbols,
    start=start_date,
    end=end_date,
    group_by="ticker",
    auto_adjust=True,
    threads=True,
)

for symb in symbols:
    single_df = raw.xs(symb, level=0, axis=1)
    single_df.to_csv("data/stockdata_small/" + symb + ".csv")


In [ ]:
path = "data/stockdata_bigcap/*.csv"
path_small = "data/stockdata_smallcap/*.csv"
paths = [path, path_small]
stockdata_dict = {}

for path in paths:
    for filename in glob.glob(path):
        single_df = pd.read_csv(filename, index_col="Date")
        single_df.index = pd.to_datetime(single_df.index)
        symbol = Path(filename).stem
        stockdata_dict[symbol] = single_df


In [ ]:
sp500 = yf.download(
    tickers="^GSPC",
    start="1980-01-02",
    end="2023-04-01",
    auto_adjust=True,
    progress=False,
)
sp500_for_func = sp500.copy()


In [ ]:
def feature_extractor(dframe):
    dframe = dframe.round(2)

    o = dframe["Open"]
    h = dframe["High"]
    l = dframe["Low"]
    c = dframe["Close"]
    v = dframe["Volume"]

    # Momentum

    PPO = ta.momentum.PercentagePriceOscillator(close=c)
    PVO = ta.momentum.PercentageVolumeOscillator(volume=v)
    ROC_10d = ta.momentum.ROCIndicator(close=c, window=10)
    ROC_60d = ta.momentum.ROCIndicator(close=c, window=60)
    RSI = ta.momentum.RSIIndicator(close=c)
    SRSI = ta.momentum.StochRSIIndicator(close=c)
    SO = ta.momentum.StochasticOscillator(high=h, close=c, low=l)
    TSI = ta.momentum.TSIIndicator(close=c)
    UO = ta.momentum.UltimateOscillator(high=h, low=l, close=c)
    WR = ta.momentum.WilliamsRIndicator(high=h, low=l, close=c)

    # Volume

    EMV = ta.volume.EaseOfMovementIndicator(high=h, low=l, volume=v)
    FI = ta.volume.ForceIndexIndicator(close=c, volume=v)
    MFI = ta.volume.MFIIndicator(high=h, low=l, close=c, volume=v)
    NVI = ta.volume.NegativeVolumeIndexIndicator(close=c, volume=v)

    # Volatility

    BB = ta.volatility.BollingerBands(close=c)
    DC = ta.volatility.DonchianChannel(high=h, low=l, close=c)
    KC = ta.volatility.KeltnerChannel(high=h, low=l, close=c)
    UI = ta.volatility.UlcerIndex(close=c)

    # Trend

    AI = ta.trend.AroonIndicator(close=c)
    CCI = ta.trend.CCIIndicator(high=h, low=l, close=c)
    KST = ta.trend.KSTIndicator(close=c)
    MACD = ta.trend.MACD(close=c)
    MI = ta.trend.MassIndex(high=h, low=l)
    STC = ta.trend.STCIndicator(close=c)
    TRIX = ta.trend.TRIXIndicator(close=c)
    VI = ta.trend.VortexIndicator(high=h, low=l, close=c)

    dframe["PPO"] = PPO.ppo_signal()
    dframe["PVO"] = PVO.pvo_signal()
    dframe["MOM10d"] = ROC_10d.roc()
    dframe["MOM60d"] = ROC_60d.roc()
    dframe["RSI"] = RSI.rsi()
    dframe["SRSI"] = SRSI.stochrsi()
    dframe["SO"] = SO.stoch_signal()
    dframe["TSI"] = TSI.tsi()
    dframe["UO"] = UO.ultimate_oscillator()
    dframe["WR"] = WR.williams_r()
    dframe["EMV"] = EMV.sma_ease_of_movement()
    dframe["FI"] = FI.force_index()
    dframe["MFI"] = MFI.money_flow_index()
    dframe["NVI"] = NVI.negative_volume_index()
    dframe["BB"] = BB.bollinger_hband_indicator()
    dframe["BBW"] = BB.bollinger_pband()
    dframe["DC"] = DC.donchian_channel_pband()
    dframe["KC"] = KC.keltner_channel_hband_indicator()
    dframe["UI"] = UI.ulcer_index()
    dframe["AI"] = AI.aroon_indicator()
    dframe["CCI"] = CCI.cci()
    dframe["KST"] = KST.kst_sig()
    dframe["MACD"] = MACD.macd_signal()
    dframe["MI"] = MI.mass_index()
    dframe["STC"] = STC.stc()
    dframe["TRIX"] = TRIX.trix()
    dframe["VI"] = VI.vortex_indicator_diff()

    dframe["BETA60d"] = (
        dframe["Close"]
        .pct_change()
        .rolling(60)
        .cov(sp500_for_func["Close"].pct_change())
    ) / (sp500_for_func["Close"].pct_change().rolling(60).var())

    dframe["BETA20d"] = (
        dframe["Close"]
        .pct_change()
        .rolling(20)
        .cov(sp500_for_func["Close"].pct_change())
    ) / (sp500_for_func["Close"].pct_change().rolling(20).var())

    dframe["BETA20d^2"] = dframe["BETA20d"] ** 2
    dframe["CHBETA"] = dframe["BETA20d"] - dframe["BETA60d"]
    dframe["VOLA20d"] = dframe["Close"].pct_change().rolling(20).std()
    dframe["VOLA60d"] = dframe["Close"].pct_change().rolling(60).std()
    dframe["CHVOLA"] = dframe["VOLA20d"] - dframe["VOLA60d"]
    dframe["MAXRET"] = dframe["Close"].pct_change().rolling(20).max()
    dframe["MINRET"] = dframe["Close"].pct_change().rolling(20).min()
    dframe["CHMOM"] = dframe["MOM10d"] - dframe["MOM60d"]

    rule = {
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Close": "last",
        "Volume": "sum",
    }

    df_ta = dframe.drop(["Open", "High", "Low", "Volume"], axis=1)

    dframe = df_ta.resample("M").last()

    dframe["RETURN"] = dframe["Close"].pct_change()

    dframe = dframe.drop(["Close"], axis=1)

    return dframe


In [ ]:
ROC10d = ta.momentum.ROCIndicator(sp500["Close"], window=10)
ROC40d = ta.momentum.ROCIndicator(sp500["Close"], window=40)

sp500.drop(["Open", "High", "Low", "Volume"], axis=1, inplace=True)
sp500["MRKTMOM10d"] = ROC10d.roc()
sp500["MRKTMOM40d"] = ROC40d.roc()
sp500["MRKTVOLA"] = sp500["Close"].pct_change().rolling(20).std()
sp500 = sp500.resample("M").last()
sp500["MRKT"] = sp500["Close"].pct_change()
sp500.drop("Close", axis=1, inplace=True)


In [ ]:
sp500_ratio = nasdaqdatalink.get(
    [
        "MULTPL/SP500_DIV_YIELD_MONTH",
        "MULTPL/SP500_EARNINGS_YIELD_MONTH",
        "MULTPL/SP500_PE_RATIO_MONTH",
    ],
    start_date="1980-01-01",
    end_date="2023-03-31",
)


In [ ]:
rule = {
    "MULTPL/SP500_DIV_YIELD_MONTH - Value": "last",
    "MULTPL/SP500_EARNINGS_YIELD_MONTH - Value": "first",
    "MULTPL/SP500_PE_RATIO_MONTH - Value": "first",
}

sp500_ratios = sp500_ratio.resample("M").apply(rule)
sp500_ratios.columns = ["DIVYIELD", "EARNYIELD", "PERATIO"]
sp500_ratios["DIVYIELD"] = sp500_ratios["DIVYIELD"].shift(1)


In [ ]:
fredrates = web.DataReader(
    ["T10Y2Y", "DTB3", "DTB6", "DPRIME", "DFF"],
    "fred",
    start="1980-01-01",
    end="2023-03-31",
)

fredrates = fredrates.resample("M").last()
fredrates["TB6TB3"] = fredrates["DTB6"] - fredrates["DTB3"]
fredrates["TB3"] = fredrates["DTB3"].diff()
fredrates["TB6"] = fredrates["DTB6"].diff()
fredrates["PRIMETB3"] = fredrates["DPRIME"] - fredrates["DTB3"]
fredrates["FFTB3"] = fredrates["DFF"] - fredrates["DTB3"]
fredrates.drop(["DTB3", "DTB6", "DPRIME", "DFF"], axis=1, inplace=True)


https://www.frbsf.org/economic-research/indicators-data/daily-news-sentiment-index/

In [ ]:
senti = pd.read_csv("data/news_sentiment_data.csv", index_col="date")
senti.index = pd.to_datetime(senti.index, format="%d.%m.%Y")
senti.columns = ["SENT1d"]
senti["SENT1d"] = pd.to_numeric(senti["SENT1d"])
senti["SENTCUM"] = senti["SENT1d"].rolling(20).sum()
senti = senti.resample("M").last()
senti["SENTMA"] = senti["SENTCUM"].rolling(3).mean()


In [ ]:
ff_dict = web.DataReader(
    "F-F_Research_Data_Factors", "famafrench", start="1980-01-02", end="2023-03-31"
)

RF = ff_dict[0].div(100)["RF"]


In [ ]:
fred_real = web.DataReader(
    ["CPIAUCSL", "UNRATE"], "fred", start="1980-01-01", end="2023-03-31"
)

fred_real["CPIAUCSL"] = fred_real["CPIAUCSL"].shift(1)
fred_real["CPIAUCSL"] = fred_real["CPIAUCSL"].pct_change()
fred_real["UNRATE"] = fred_real["UNRATE"].shift(1)
fred_real.columns = ["CPI", "UNRATE"]

fred_real = fred_real.resample("M").last()


In [ ]:
macros = sp500.join(sp500_ratios)
macros = macros.join(fredrates)
macros = macros.join(senti)
macros = macros.join(fred_real)
macros["RF"] = RF.values


In [ ]:
stockdata_feature_dict = {}
for symbol in stockdata_dict:
    dframe = stockdata_dict[symbol]
    dframe_feature = feature_extractor(dframe)
    dframe_feature = dframe_feature.join(macros)
    dframe_feature["EXCESS_RETURN"] = dframe_feature["RETURN"] - dframe_feature["RF"]
    dframe_feature["EXCESS_RETURN_T+1"] = dframe_feature["EXCESS_RETURN"].shift(-1)
    dframe_feature.drop(["EXCESS_RETURN"], inplace=True, axis=1)
    dframe_feature.dropna(axis=0, inplace=True)
    stockdata_feature_dict[symbol] = dframe_feature


In [ ]:
for symb in stockdata_feature_dict:
    single_df = stockdata_feature_dict[symb]
    single_df.to_csv("data/stockdata_feature_full/" + symb + ".csv")


In [ ]:
path = "data/stockdata_feature_full/*.csv"
stockdata_feature_dict = {}
for filename in glob.glob(path):
    single_df = pd.read_csv(filename, index_col="Date")
    single_df.index = pd.to_datetime(single_df.index)
    symbol = Path(filename).stem
    stockdata_feature_dict[symbol] = single_df


In [2]:
path_big = "data/stockdata_bigcap/*.csv"
path_small = "data/stockdata_smallcap/*.csv"

stockdata_feature_small_dict = {}
stockdata_feature_big_dict = {}

for filename in glob.glob(path_small):
    symbol = Path(filename).stem
    single_df = pd.read_csv("data/stockdata_feature_full/" + symbol +".csv", index_col="Date")
    stockdata_feature_small_dict[symbol] = single_df

for filename in glob.glob(path_big):
    symbol = Path(filename).stem
    single_df = pd.read_csv("data/stockdata_feature_full/" + symbol +".csv", index_col="Date")
    stockdata_feature_big_dict[symbol] = single_df

In [3]:
panel_bigcap = pd.concat([frame for frame in stockdata_feature_big_dict.values()])
panel_smallcap = pd.concat([frame for frame in stockdata_feature_small_dict.values()])

In [4]:
micro_cols = [
    "PPO","PVO","MOM10d","MOM60d","RSI","SRSI","SO","TSI","UO","WR","EMV","FI","MFI","NVI","BB","BBW",
    "DC","KC","UI","AI","CCI","KST","MACD","MI","STC","TRIX","VI","BETA60d","BETA20d","BETA20d^2",
    "CHBETA","VOLA20d","VOLA60d","CHVOLA","MAXRET","MINRET","CHMOM","RETURN"
]

macro_cols = [
    "MRKTMOM10d","MRKTMOM40d","MRKTVOLA","MRKT","DIVYIELD","EARNYIELD","PERATIO","T10Y2Y","TB6TB3",
    "TB3","TB6","PRIMETB3","FFTB3","SENT1d","SENTCUM","SENTMA","CPI","UNRATE","RF"
]

In [7]:
for frame in (panel_bigcap, panel_smallcap):
    for col_macro in macro_cols:
        for col_micro in micro_cols:
            frame[col_micro +"x"+ col_macro] = frame[col_macro]*frame[col_micro]
    frame.drop(macro_cols, axis=1, inplace=True)

C:\Users\Tim\AppData\Local\Temp\ipykernel_1072\3936213886.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  frame[col_micro +"x"+ col_macro] = frame[col_macro]*frame[col_micro]
C:\Users\Tim\AppData\Local\Temp\ipykernel_1072\3936213886.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  frame[col_micro +"x"+ col_macro] = frame[col_macro]*frame[col_micro]
C:\Users\Tim\AppData\Local\Temp\ipykernel_1072\3936213886.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

In [9]:
panel_bigcap.to_csv("data/panel_bigcap_x")
panel_smallcap.to_csv("data/panel_smallcap_x")

In [ ]:
panel = pd.concat([frame for frame in stockdata_feature_dict.values()])


In [ ]:
cols_to_scale = [
    "PPO",
    "PVO",
    "MOM10d",
    "MOM60d",
    "RSI",
    "SRSI",
    "SO",
    "TSI",
    "UO",
    "WR",
    "EMV",
    "FI",
    "MFI",
    "NVI",
    "BB",
    "BBW",
    "DC",
    "KC",
    "UI",
    "AI",
    "CCI",
    "KST",
    "MACD",
    "MI",
    "STC",
    "TRIX",
    "VI",
    "BETA60d",
    "BETA20d",
    "BETA20d^2",
    "CHBETA",
    "VOLA20d",
    "VOLA60d",
    "CHVOLA",
    "MAXRET",
    "MINRET",
    "CHMOM",
    "RETURN",
]

for date in panel.index.unique():
    for feature in cols_to_scale:
        crosssec_min = panel.loc[date, feature].values.min()
        crosssec_max = panel.loc[date, feature].values.max()
        panel.loc[date, feature] = panel.loc[date, feature].apply(
            lambda x: 2 * (x - crosssec_min) / (crosssec_max - crosssec_min) - 1
        )


In [ ]:
macro_cols = [
    "MRKTMOM10d",
    "MRKTMOM40d",
    "MRKTVOLA",
    "MRKT",
    "DIVYIELD",
    "EARNYIELD",
    "PERATIO",
    "T10Y2Y",
    "TB6TB3",
    "TB3",
    "TB6",
    "PRIMETB3",
    "FFTB3",
    "SENT1d",
    "SENTCUM",
    "SENTMA",
    "CPI",
    "UNRATE",
    "RF",
]

for col in macro_cols:
    colmin = panel[col].values.min()
    colmax = panel[col].values.max()
    panel[col] = panel[col].apply(lambda x: 2 * (x - colmin) / (colmax - colmin) - 1)


In [ ]:
panel.to_csv("data/panel_full.csv")

In [ ]:
path_big = "data/stockdata_bigcap/*.csv"
path_small = "data/stockdata_smallcap/*.csv"

stockdata_feature_small_dict = {}
stockdata_feature_big_dict = {}

for filename in glob.glob(path_small):
    symbol = Path(filename).stem
    single_df = pd.read_csv("data/stockdata_feature_full/" + symbol +".csv")
    stockdata_feature_small_dict[symbol] = single_df

for filename in glob.glob(path_big):
    symbol = Path(filename).stem
    single_df = pd.read_csv("data/stockdata_feature_full/" + symbol +".csv")
    stockdata_feature_big_dict[symbol] = single_df

panel_bigcap = pd.concat([frame for frame in stockdata_feature_big_dict.values()])
panel_smallcap = pd.concat([frame for frame in stockdata_feature_small_dict.values()])

In [ ]:
panel_bigcap.to_csv("data/panel_bigcap.csv")
panel_smallcap.to_csv("data/panel_smallcap.csv")